In [16]:
import pandas as pd
import cvxpy as cp
import numpy as np

In [3]:
df = pd.read_csv("~/Desktop/OSU/OSU Senior/Fall/ISE 3230/poi_and_kiosk_by_ct_data.csv")

In [8]:
df['COUNTYFP'].unique()

category_map = {
    5 : 1500, # Bronx
    47 : 1000, # Brooklyn
    61 : 1250, # Manhattan
    81 : 1500, # Quenns
    85 : 1100, # Staten Island
}

df['Cost'] = df['COUNTYFP'].map(category_map)

In [35]:
geoid = df['GEOID']
kiosks=df['kiosks'].to_numpy()
pop=df['population'].to_numpy()
pois=df['pois'].to_numpy()
cost=df['Cost'].to_numpy()
borough=df['COUNTYFP']

pois = np.nan_to_num(pois, nan=1, posinf=1, neginf=1)

In [46]:
n = len(geoid)

array([1.3 , 0.45, 0.5 , ..., 0.2 , 0.15, 1.4 ])

In [51]:
X = cp.Variable(n, integer=True)
Z = cp.Variable(n, boolean=True)

obj_fun = cp.sum(cp.multiply(cost, X))

constraints = []
constraints.append(X>=0)
constraints.append(X<=2000)
constraints.append(X <= pois/10 * 2000)
constraints.append(X <= pop/5000 * 2000)
constraints.append(pop/5000<=kiosks+X)


problem = cp.Problem(cp.Minimize(obj_fun), constraints)
problem.solve(solver=cp.GUROBI, verbose=True)


print("Solver status:", problem.status)
print("X values:", X.value)
print("Objective Function", obj_fun.value)


                                     CVXPY                                     
                                     v1.5.3                                    
(CVXPY) Nov 25 12:31:28 AM: Your problem has 2327 variables, 11635 constraints, and 0 parameters.
(CVXPY) Nov 25 12:31:28 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 25 12:31:28 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 25 12:31:28 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 25 12:31:28 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Nov 25 12:31:28 AM: Compiling problem (target solver=GUROB

In [57]:
sum(X.value)
for b in [5, 47, 61, 81, 85]:  # Adjust number of boroughs as needed
    # Sum kiosks added in this borough
    kiosks_in_borough = np.sum(X.value[borough == b])
    print(f"Borough {b} : Kiosks added = {kiosks_in_borough}")

Borough 5 : Kiosks added = 356.0
Borough 47 : Kiosks added = 719.0
Borough 61 : Kiosks added = 142.0
Borough 81 : Kiosks added = 641.0
Borough 85 : Kiosks added = 130.0
